In [3]:
import pandas as pd
from pandas import Series,DataFrame
Critics = pd.read_csv('Critics.csv', header = None, names = ["Author","Score","Date","Comment","Game"])
Critics["Type"] = "Expert"
Users = pd.read_csv('Users.csv', header = None, names = ["Author","Score","Date","Comment","Game"])
Users["Type"] = "User"
CriticsW = pd.read_csv('CriticsWon.csv', header = None, names = ["Author","Score","Date","Comment","Game"])
CriticsW["Type"] = "Expert"
UsersW = pd.read_csv('UsersWon.csv', header = None, names = ["Author","Score","Date","Comment","Game"])
UsersW["Type"] = "User"
CriticsJ = pd.read_csv('CriticsJ.csv', header = None, names = ["Author","Score","Date","Comment","Game"])
CriticsJ["Type"] = "Expert"
UsersJ = pd.read_csv('UsersJ.csv', header = None, names = ["Author","Score","Date","Comment","Game"])
UsersJ["Type"] = "User"


Full = pd.concat([Critics,Users,CriticsW,UsersW,CriticsJ,UsersJ])
gamelist = {"Overwatch":["A","FPS","H"],"Halo MCC":["A","FPS","H"],"Borderlands 2":["A","RPG","H"],"Saints Row 4":["A","AA","H"],
            "The division":["A","FPS","H"],"Bioshock":["A","FPS","H"],"God of War":["A","AA","H"],"Black Ops":["A","FPS","H"],
            "Destiny":["A","FPS","H"],"Zelda":["A","AA","H"],"Last of Us":["A","AA","H"],"Skyrim":["A","RPG","H"],"Oblivion":["A","RPG","H"],
            "Shadow of War":["A","AA","M"],"Dead Rising 4":["A","AA","M"],"Cuphead":["I","R+G","M"],"Overcooked":["I","Puz","M"],
            "Papers Please":["I","Puz","M"],"Just Cause":["A","AA","M"],"Burnout":["A","Rac","M"],"Dark Souls":["A","AA","M"],
            "Dishonored":["A","AA","M"],"Dying Light":["A","AA","M"],"Madden 19":["A","S","M"],"Sunset Overdrive":["A","AA","L"],
            "NHL 18":["A","S","L"],"Quantum Break":["A","FPS","L"],"Dead Island Riptide":["A","AA","L"],"No Mans Sky":["A","AA","L"],
            "Witness":["I","Puz","L"],"Firewatch":["I","Puz","L"],"ARMA":["I","FPS","L"],"Skylines":["I","Puz","L"],
            "Stardew Valley":["A","Puz","M"]}
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

def sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)
    return snt["compound"]
def sentiment_rating(snt_score):
    if snt_score < -.1:
        return "Neg"
    elif snt_score > .1:
        return "Pos"
    else:
        return "Neu"
def developer(game):
    return gamelist[game][0]
def genre(game):
    return gamelist[game][1]
def tier(game):
    return gamelist[game][2]
Full["SentScore"]= Full["Comment"].map(sentiment_scores)
Full["SentRate"]= Full["SentScore"].map(sentiment_rating)
Full["Developer"] = Full["Game"].map(developer)
Full["Genre"] = Full["Game"].map(genre)
Full["Tier"] = Full["Game"].map(tier)

In [2]:
doc_complete = []
for i in range(1,len(Full)):
    doc_complete.append(Full.iloc[i][3])


In [5]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete]  

In [4]:
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]



In [5]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=4, id2word = dictionary, passes=50)

print(ldamodel.print_topics(num_topics=4, num_words=4))


[(0, '0.022*"expand" + 0.010*"review" + 0.009*"destiny" + 0.009*"2"'), (1, '0.037*"de" + 0.033*"que" + 0.020*"la" + 0.019*"e"'), (2, '0.009*"jogo" + 0.007*"и" + 0.005*"hello" + 0.005*"um"'), (3, '0.072*"game" + 0.010*"one" + 0.009*"story" + 0.008*"like"')]


In [6]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=50)

print(ldamodel.print_topics(num_topics=10, num_words=3))


[(0, '0.031*"best" + 0.029*"game" + 0.017*"god"'), (1, '0.070*"game" + 0.017*"story" + 0.013*"great"'), (2, '0.023*"pc" + 0.018*"mod" + 0.009*"game"'), (3, '0.060*"expand" + 0.035*"review" + 0.034*"destiny"'), (4, '0.042*"de" + 0.037*"que" + 0.025*"e"'), (5, '0.026*"halo" + 0.012*"scroll" + 0.012*"elder"'), (6, '0.016*"2" + 0.014*"like" + 0.013*"zombie"'), (7, '0.013*"sony" + 0.009*"nintendo" + 0.008*"fighting"'), (8, '0.027*"city" + 0.017*"skyrim" + 0.011*"row"'), (9, '0.095*"game" + 0.010*"play" + 0.010*"it"')]


In [7]:
print(ldamodel.print_topics(num_topics=10, num_words=4))

[(0, '0.031*"best" + 0.029*"game" + 0.017*"god" + 0.017*"war"'), (1, '0.070*"game" + 0.017*"story" + 0.013*"great" + 0.011*"expand"'), (2, '0.023*"pc" + 0.018*"mod" + 0.009*"game" + 0.008*"run"'), (3, '0.060*"expand" + 0.035*"review" + 0.034*"destiny" + 0.024*"view"'), (4, '0.042*"de" + 0.037*"que" + 0.025*"e" + 0.023*"la"'), (5, '0.026*"halo" + 0.012*"scroll" + 0.012*"elder" + 0.011*"и"'), (6, '0.016*"2" + 0.014*"like" + 0.013*"zombie" + 0.012*"3"'), (7, '0.013*"sony" + 0.009*"nintendo" + 0.008*"fighting" + 0.008*"sim"'), (8, '0.027*"city" + 0.017*"skyrim" + 0.011*"row" + 0.011*"saint"'), (9, '0.095*"game" + 0.010*"play" + 0.010*"it" + 0.010*"one"')]


In [8]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=50)

print(ldamodel.print_topics(num_topics=15, num_words=4))


[(0, '0.040*"zelda" + 0.034*"city" + 0.021*"breath" + 0.018*"wild"'), (1, '0.070*"game" + 0.013*"like" + 0.013*"expand" + 0.009*"play"'), (2, '0.014*"arma" + 0.013*"jogo" + 0.011*"e" + 0.010*"de"'), (3, '0.019*"u" + 0.014*"new" + 0.011*"last" + 0.010*"dog"'), (4, '0.014*"planet" + 0.009*"system" + 0.006*"lie" + 0.006*"run"'), (5, '0.043*"game" + 0.012*"mod" + 0.009*"buy" + 0.007*"money"'), (6, '0.086*"game" + 0.020*"best" + 0.020*"story" + 0.016*"one"'), (7, '0.018*"ops" + 0.014*"3" + 0.012*"row" + 0.012*"saint"'), (8, '0.038*"de" + 0.035*"que" + 0.033*"expand" + 0.025*"la"'), (9, '0.034*"call" + 0.025*"duty" + 0.011*"uncharted" + 0.010*"historia"')]


In [10]:
import gensim
from gensim import corpora
for g in gamelist.keys():
    Gamedf = Full[Full["Game"] == g]
    doc_complete = []
    for i in range(1,len(Gamedf)):
        doc_complete.append(Gamedf.iloc[i][3])
    doc_clean = [clean(doc).split() for doc in doc_complete]
    dictionary = corpora.Dictionary(doc_clean)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

    # Creating the object for LDA model using gensim library
    Lda = gensim.models.ldamodel.LdaModel
    # Running and Trainign LDA model on the document term matrix.
    ldamodel = Lda(doc_term_matrix, num_topics=4, id2word = dictionary, passes=30)
    print(g)
    print(ldamodel.print_topics(num_topics=4, num_words=4))

Overwatch
[(0, '0.015*"que" + 0.014*"de" + 0.009*"e" + 0.007*"la"'), (1, '0.052*"game" + 0.013*"fun" + 0.011*"blizzard" + 0.010*"expand"'), (2, '0.056*"game" + 0.016*"team" + 0.014*"overwatch" + 0.011*"character"'), (3, '0.060*"game" + 0.016*"play" + 0.014*"fun" + 0.012*"overwatch"')]
Halo MCC
[(0, '0.019*"game" + 0.018*"halo" + 0.013*"multiplayer" + 0.010*"de"'), (1, '0.044*"game" + 0.042*"halo" + 0.013*"2" + 0.012*"4"'), (2, '0.036*"halo" + 0.017*"collection" + 0.016*"master" + 0.015*"chief"'), (3, '0.049*"game" + 0.029*"halo" + 0.011*"one" + 0.011*"multiplayer"')]
Borderlands 2
[(0, '0.036*"game" + 0.023*"borderland" + 0.015*"2" + 0.012*"gun"'), (1, '0.059*"game" + 0.018*"like" + 0.012*"borderland" + 0.011*"first"'), (2, '0.021*"game" + 0.011*"one" + 0.009*"really" + 0.008*"borderland"'), (3, '0.046*"game" + 0.017*"borderland" + 0.012*"great" + 0.011*"first"')]
Saints Row 4
[(0, '0.058*"game" + 0.030*"saint" + 0.030*"row" + 0.014*"fun"'), (1, '0.044*"game" + 0.024*"saint" + 0.023*"r

Witness
[(0, '0.042*"game" + 0.025*"puzzle" + 0.011*"like" + 0.007*"witness"'), (1, '0.015*"game" + 0.015*"witness" + 0.014*"puzzle" + 0.008*"review"'), (2, '0.045*"game" + 0.029*"puzzle" + 0.007*"solve" + 0.007*"great"'), (3, '0.040*"game" + 0.036*"puzzle" + 0.011*"like" + 0.011*"witness"')]
Firewatch
[(0, '0.024*"expand" + 0.020*"game" + 0.017*"review" + 0.016*"click"'), (1, '0.046*"game" + 0.018*"firewatch" + 0.016*"story" + 0.009*"like"'), (2, '0.059*"game" + 0.012*"story" + 0.009*"firewatch" + 0.008*"really"'), (3, '0.024*"game" + 0.022*"story" + 0.010*"walking" + 0.009*"good"')]
ARMA
[(0, '0.025*"arma" + 0.021*"game" + 0.010*"3" + 0.010*"great"'), (1, '0.039*"game" + 0.012*"arma" + 0.009*"player" + 0.007*"series"'), (2, '0.047*"game" + 0.019*"arma" + 0.011*"3" + 0.011*"content"'), (3, '0.032*"game" + 0.026*"arma" + 0.009*"3" + 0.009*"better"')]
Skylines
[(0, '0.061*"city" + 0.025*"game" + 0.019*"skyline" + 0.018*"simcity"'), (1, '0.049*"game" + 0.028*"city" + 0.010*"skyline" + 0.

In [11]:
import gensim
from gensim import corpora
for g in ["User","Expert"]:
    Gamedf = Full[Full["Type"] == g]
    doc_complete = []
    for i in range(1,len(Gamedf)):
        doc_complete.append(Gamedf.iloc[i][3])
    doc_clean = [clean(doc).split() for doc in doc_complete]
    dictionary = corpora.Dictionary(doc_clean)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

    # Creating the object for LDA model using gensim library
    Lda = gensim.models.ldamodel.LdaModel
    # Running and Trainign LDA model on the document term matrix.
    ldamodel = Lda(doc_term_matrix, num_topics=4, id2word = dictionary, passes=30)
    print(g)
    print(ldamodel.print_topics(num_topics=4, num_words=4))

User
[(0, '0.071*"game" + 0.010*"like" + 0.009*"story" + 0.009*"one"'), (1, '0.016*"mod" + 0.015*"zombie" + 0.011*"skyrim" + 0.010*"oblivion"'), (2, '0.032*"de" + 0.028*"que" + 0.019*"e" + 0.017*"la"'), (3, '0.041*"expand" + 0.021*"review" + 0.014*"view" + 0.013*"spoiler"')]
Expert
[(0, '0.008*"paradise" + 0.007*"burnout" + 0.006*"god" + 0.006*"war"'), (1, '0.024*"game" + 0.010*"best" + 0.010*"one" + 0.008*"city"'), (2, '0.035*"game" + 0.008*"new" + 0.007*"one" + 0.007*"it’s"'), (3, '0.029*"game" + 0.010*"one" + 0.008*"gameplay" + 0.007*"experience"')]


In [13]:
import gensim
from gensim import corpora
for g in ["I","A"]:
    Gamedf = Full[Full["Developer"] == g]
    doc_complete = []
    for i in range(1,len(Gamedf)):
        doc_complete.append(Gamedf.iloc[i][3])
    doc_clean = [clean(doc).split() for doc in doc_complete]
    dictionary = corpora.Dictionary(doc_clean)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

    # Creating the object for LDA model using gensim library
    Lda = gensim.models.ldamodel.LdaModel
    # Running and Trainign LDA model on the document term matrix.
    ldamodel = Lda(doc_term_matrix, num_topics=4, id2word = dictionary, passes=30)
    print(g)
    print(ldamodel.print_topics(num_topics=4, num_words=4))

I
[(0, '0.005*"arma" + 0.004*"de" + 0.003*"que" + 0.003*"see"'), (1, '0.067*"game" + 0.011*"like" + 0.008*"story" + 0.008*"great"'), (2, '0.035*"city" + 0.029*"game" + 0.012*"simcity" + 0.010*"skyline"'), (3, '0.018*"expand" + 0.012*"arma" + 0.011*"game" + 0.010*"review"')]
A
[(0, '0.048*"game" + 0.013*"story" + 0.011*"like" + 0.009*"expand"'), (1, '0.061*"game" + 0.026*"best" + 0.019*"ever" + 0.017*"one"'), (2, '0.064*"game" + 0.015*"expand" + 0.011*"review" + 0.008*"hour"'), (3, '0.032*"de" + 0.028*"que" + 0.019*"e" + 0.018*"la"')]


In [6]:
import gensim
from gensim import corpora
for g in ["AA","FPS","RPG","Puz","S"]:
    Gamedf = Full[Full["Genre"] == g]
    doc_complete = []
    for i in range(1,len(Gamedf)):
        doc_complete.append(Gamedf.iloc[i][3])
    doc_clean = [clean(doc).split() for doc in doc_complete]
    dictionary = corpora.Dictionary(doc_clean)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

    # Creating the object for LDA model using gensim library
    Lda = gensim.models.ldamodel.LdaModel
    # Running and Trainign LDA model on the document term matrix.
    ldamodel = Lda(doc_term_matrix, num_topics=4, id2word = dictionary, passes=30)
    print(g)
    print(ldamodel.print_topics(num_topics=4, num_words=4))

AA
[(0, '0.030*"expand" + 0.023*"review" + 0.018*"view" + 0.017*"contains"'), (1, '0.031*"de" + 0.029*"que" + 0.020*"e" + 0.018*"la"'), (2, '0.069*"game" + 0.010*"one" + 0.009*"story" + 0.009*"like"'), (3, '0.055*"game" + 0.007*"like" + 0.007*"man" + 0.007*"soul"')]
FPS
[(0, '0.025*"expand" + 0.017*"review" + 0.015*"view" + 0.013*"contains"'), (1, '0.021*"de" + 0.017*"que" + 0.015*"destiny" + 0.012*"la"'), (2, '0.060*"game" + 0.010*"story" + 0.009*"one" + 0.009*"good"'), (3, '0.018*"game" + 0.016*"cod" + 0.015*"call" + 0.014*"duty"')]
RPG
[(0, '0.048*"game" + 0.019*"borderland" + 0.014*"expand" + 0.011*"2"'), (1, '0.025*"game" + 0.010*"get" + 0.007*"system" + 0.006*"skyrim"'), (2, '0.052*"game" + 0.013*"expand" + 0.011*"oblivion" + 0.010*"quest"'), (3, '0.048*"game" + 0.028*"mod" + 0.010*"paid" + 0.009*"like"')]
Puz
[(0, '0.067*"game" + 0.009*"story" + 0.008*"like" + 0.008*"one"'), (1, '0.018*"game" + 0.009*"firewatch" + 0.006*"one" + 0.006*"story"'), (2, '0.004*"la" + 0.003*"un" + 0.0